In [2]:
# 런타임 초기화 후 제일 먼저
import gc
gc.collect()

0

In [3]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce

In [7]:
train_7 = pd.read_csv('train_7_마케팅정보_98이상제거.csv')
segment = pd.read_csv("Segment_병합.csv")

In [8]:
# 1️⃣ concat으로 segment 컬럼 붙이기
train_full = pd.concat([train_7, segment['Segment']], axis=1)

In [9]:
train_full

,기준년월,ID,컨택건수_카드론_TM_B0M,컨택건수_이용유도_TM_B0M,컨택건수_이용유도_LMS_B0M,컨택건수_이용유도_EM_B0M,컨택건수_이용유도_청구서_B0M,컨택건수_이용유도_인터넷_B0M,컨택건수_카드론_TM_R6M,컨택건수_리볼빙_TM_R6M,...,컨택건수_이용유도_LMS_R6M,컨택건수_이용유도_EM_R6M,컨택건수_카드론_청구서_R6M,컨택건수_CA_청구서_R6M,컨택건수_이용유도_청구서_R6M,컨택건수_이용유도_인터넷_R6M,컨택건수_채권_R6M,캠페인접촉건수_R12M,캠페인접촉일수_R12M,Segment
0,201807,TRAIN_000000,0,0,2,13,0,0,0,0,...,7,57,0,0,0,2,0,1회 이상,1일 이상,D
1,201807,TRAIN_000001,3,0,0,0,0,1,9,0,...,5,2,2,2,1,6,0,15회 이상,10일 이상,E
2,201807,TRAIN_000002,0,0,0,0,0,0,0,0,...,0,12,0,0,1,3,0,1회 이상,1일 이상,C
3,201807,TRAIN_000003,0,0,2,13,1,0,0,0,...,9,35,0,0,2,0,0,1회 이상,1일 이상,D
4,201807,TRAIN_000004,0,0,3,0,1,0,0,0,...,16,0,0,0,4,1,0,1회 이상,1일 이상,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,1회 이상,1일 이상,E
2399996,201812,TRAIN_399996,0,0,2,13,0,0,9,0,...,12,58,0,0,0,0,0,20회 이상,15일 이상,D
2399997,201812,TRAIN_399997,0,0,0,0,1,0,0,0,...,0,0,0,0,6,0,0,1회 이상,1일 이상,C
2399998,201812,TRAIN_399998,0,0,0,0,0,0,0,0,...,0,0,0,0,4,1,0,1회 이상,1일 이상,E


In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [11]:
target = train_full['Segment']
X = train_full.drop(columns=['ID', 'Segment'])

In [12]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [13]:
from sklearn.preprocessing import LabelEncoder

# 문자열 target → 숫자형으로 변환
le = LabelEncoder()
target_encoded = le.fit_transform(target)

print("\n✅ 타겟 라벨 인코딩 완료")
print(pd.Series(target_encoded).value_counts())


✅ 타겟 라벨 인코딩 완료
4    1922052
3     349242
2     127590
0        972
1        144
Name: count, dtype: int64


In [14]:
corrs = X_encoded.corrwith(pd.Series(target_encoded))
corrs_sorted = corrs.abs().sort_values(ascending=False)

print("\n✅ 상관계수 (절대값 내림차순):")
print(corrs_sorted)


✅ 상관계수 (절대값 내림차순):
컨택건수_이용유도_TM_R6M       1.490369e-01
컨택건수_이용유도_EM_R6M       1.333041e-01
컨택건수_이용유도_EM_B0M       1.275837e-01
컨택건수_이용유도_청구서_B0M      8.147784e-02
컨택건수_이용유도_TM_B0M       6.990714e-02
컨택건수_이용유도_인터넷_R6M      6.255555e-02
캠페인접촉일수_R12M_5일 이상     4.160972e-02
컨택건수_이용유도_LMS_B0M      4.124644e-02
캠페인접촉건수_R12M_5회 이상     3.957971e-02
캠페인접촉일수_R12M_1일 이상     3.900636e-02
캠페인접촉건수_R12M_1회 이상     3.833582e-02
컨택건수_보험_TM_R6M         2.971020e-02
컨택건수_이용유도_청구서_R6M      2.883512e-02
컨택건수_카드론_TM_R6M        2.419552e-02
컨택건수_카드론_LMS_R6M       2.245315e-02
컨택건수_채권_R6M            2.200994e-02
컨택건수_CA_청구서_R6M        1.966827e-02
컨택건수_이용유도_LMS_R6M      1.697038e-02
컨택건수_CA_LMS_R6M        1.674646e-02
컨택건수_이용유도_인터넷_B0M      1.215455e-02
컨택건수_리볼빙_TM_R6M        8.355822e-03
컨택건수_카드론_TM_B0M        6.422731e-03
컨택건수_부대서비스_TM_R6M      6.387984e-03
컨택건수_카드론_청구서_R6M       4.350064e-03
캠페인접촉건수_R12M_20회 이상    4.173249e-03
캠페인접촉일수_R12M_20일 이상    3.375961e-03
캠페인접촉건수_R12M_15회 이상    1.497992e-03
캠페인접촉일수_

In [16]:
corrs_sorted = corrs.sort_values(ascending=False)
print(corrs_sorted.round(3))

컨택건수_이용유도_TM_R6M       0.149
컨택건수_이용유도_TM_B0M       0.070
컨택건수_이용유도_인터넷_R6M      0.063
캠페인접촉일수_R12M_5일 이상     0.042
캠페인접촉건수_R12M_5회 이상     0.040
컨택건수_보험_TM_R6M         0.030
컨택건수_카드론_TM_R6M        0.024
컨택건수_카드론_LMS_R6M       0.022
컨택건수_CA_LMS_R6M        0.017
컨택건수_이용유도_인터넷_B0M      0.012
컨택건수_리볼빙_TM_R6M        0.008
컨택건수_카드론_TM_B0M        0.006
컨택건수_부대서비스_TM_R6M      0.006
캠페인접촉건수_R12M_15회 이상    0.001
캠페인접촉건수_R12M_25회 이상    0.000
기준년월                  -0.000
캠페인접촉일수_R12M_15일 이상   -0.000
캠페인접촉일수_R12M_20일 이상   -0.003
캠페인접촉건수_R12M_20회 이상   -0.004
컨택건수_카드론_청구서_R6M      -0.004
컨택건수_이용유도_LMS_R6M     -0.017
컨택건수_CA_청구서_R6M       -0.020
컨택건수_채권_R6M           -0.022
컨택건수_이용유도_청구서_R6M     -0.029
캠페인접촉건수_R12M_1회 이상    -0.038
캠페인접촉일수_R12M_1일 이상    -0.039
컨택건수_이용유도_LMS_B0M     -0.041
컨택건수_이용유도_청구서_B0M     -0.081
컨택건수_이용유도_EM_B0M      -0.128
컨택건수_이용유도_EM_R6M      -0.133
dtype: float64
